<a href="https://colab.research.google.com/github/sayyamalam/sign-language-recognition/blob/main/download_raw_selected50_vids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Colab: Google Drive mounten ---
from google.colab import drive
drive.mount('/content/drive')

# --- Nützliche Tools installieren/prüfen ---
!pip -q install yt-dlp tqdm
# ffmpeg ist in Colab normalerweise vorinstalliert; zur Sicherheit:
!apt -yq install ffmpeg >/dev/null


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.2 MB/s eta 0:00:00




In [ ]:
import os
import re
import json
import time
import glob
import logging
import socket
import subprocess
import threading
from pathlib import Path
from datetime import datetime, timezone
from typing import Dict, List, Tuple
from collections import deque, defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

# =========================
# CONFIG
# =========================
REPO_DIR        = "/content/sign-language-recognition"
SELECTED_DIR    = f"{REPO_DIR}/meta/selected50"
COOKIES_PATH    = f"{SELECTED_DIR}/cookies.txt"

DRIVE_ROOT      = "/content/drive/MyDrive/msasl_clips"
TMP_ROOT        = "/content/msasl_tmp"   # volles Rohvideo (lokal/SSD, temporär)
TMP_YID_DIR     = f"{TMP_ROOT}/_yid"     # ein Temp pro YTID (wichtig!)

LOG_DIR         = f"{DRIVE_ROOT}/logs"
STATE_DIR       = f"{LOG_DIR}/state"
PROCESSED_FILE  = f"{STATE_DIR}/processed_ids.txt"
FAILED_FILE     = f"{STATE_DIR}/failed_ids.txt"
PROGRESS_JSON   = f"{STATE_DIR}/progress.json"
MAIN_LOG_FILE   = f"{LOG_DIR}/downloader.log"

# Performance
YTDLP_RETRIES        = 3
FRAGMENT_RETRIES     = 3
MAX_CLIP_LEN_S       = 30.0
CONCURRENT_FRAGMENTS = "8"
HTTP_CHUNK_SIZE      = "10M"

# Parallelität: # gleichzeitige YTIDs
N_WORKERS            = 2

USE_PROXY = False
PROXY_URL = None

LABEL_NAMES: Dict[int, str] = {}
log_lock = threading.Lock()
state_lock = threading.Lock()

# =========================
# UTIL: Ordner & Logging
# =========================
def ensure_dirs():
    for d in [DRIVE_ROOT, LOG_DIR, STATE_DIR, TMP_ROOT, TMP_YID_DIR]:
        Path(d).mkdir(parents=True, exist_ok=True)
    for split in ["train", "val", "test"]:
        Path(f"{DRIVE_ROOT}/{split}").mkdir(parents=True, exist_ok=True)

def setup_logging():
    ensure_dirs()
    logger = logging.getLogger("msasl_downloader")
    logger.setLevel(logging.INFO)
    logger.handlers.clear()

    fh = logging.FileHandler(MAIN_LOG_FILE, encoding="utf-8")
    fh.setLevel(logging.INFO)
    fmt = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
    fh.setFormatter(fmt)
    logger.addHandler(fh)

    sh = logging.StreamHandler()
    sh.setLevel(logging.INFO)
    console_fmt = logging.Formatter("%(message)s")
    sh.setFormatter(console_fmt)
    logger.addHandler(sh)

    logger.info("🚀 MSASL selected50 Downloader gestartet")
    logger.info(f"📁 Repo:      {REPO_DIR}")
    logger.info(f"📂 Selected:  {SELECTED_DIR}")
    logger.info(f"💾 Drive:     {DRIVE_ROOT}")
    logger.info(f"⚡ TempDir:   {TMP_ROOT}")
    logger.info(f"🍪 Cookies:   {COOKIES_PATH} ({'✅' if os.path.exists(COOKIES_PATH) else '❌'})")
    logger.info(f"🧵 Workers:   {N_WORKERS}")
    logger.info("="*60)
    return logger

def safe_log(logger, message, level="info"):
    with log_lock:
        getattr(logger, level, logger.info)(message)

def utcnow_iso():
    return datetime.now(timezone.utc).isoformat()

# =========================
# STATE / RESUME
# =========================
def read_lines(path: str) -> set:
    if not os.path.exists(path):
        return set()
    with open(path, "r", encoding="utf-8") as f:
        return set([ln.strip() for ln in f if ln.strip()])

def append_line(path: str, line: str):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with state_lock:
        with open(path, "a", encoding="utf-8") as f:
            f.write(line + "\n")

def atomic_write_json(path: str, payload: Dict):
    tmp = path + ".tmp"
    Path(tmp).parent.mkdir(parents=True, exist_ok=True)
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, path)

# =========================
# DATEN
# =========================
def _normalize_url(u: str) -> str:
    u = u.strip()
    if u.startswith("www."):
        u = "https://" + u
    return u

def load_label_names():
    global LABEL_NAMES
    try:
        for path in [
            f"{REPO_DIR}/meta/labels.json",
            f"{REPO_DIR}/labels.json",
            f"{SELECTED_DIR}/labels.json"
        ]:
            if os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                if isinstance(data, dict):
                    LABEL_NAMES = {int(k): v for k, v in data.items()}
                elif isinstance(data, list):
                    LABEL_NAMES = {i: name for i, name in enumerate(data)}
                break
    except Exception:
        pass

def get_label_display(label: int) -> str:
    if label in LABEL_NAMES:
        return f"{label} ({LABEL_NAMES[label]})"
    return str(label)

def read_selected50_entries(selected_dir: str) -> List[Dict]:
    json_paths = sorted(glob.glob(os.path.join(selected_dir, "*.json")))
    if not json_paths:
        raise FileNotFoundError(f"Keine JSONs in {selected_dir} gefunden.")
    all_entries = []
    for jp in json_paths:
        name = os.path.basename(jp).lower()
        if   "train" in name: split = "train"
        elif "val"   in name: split = "val"
        elif "test"  in name: split = "test"
        else: continue
        with open(jp, "r", encoding="utf-8") as f:
            data = json.load(f)
        for e in data:
            e["split"] = split
            e["url"] = _normalize_url(e.get("url",""))
            all_entries.append(e)
    return all_entries

# =========================
# NAMING
# =========================
def youtube_id_from_url(url: str) -> str:
    m = re.search(r"(?:v=|/shorts/|youtu\.be/)([A-Za-z0-9_\-]{6,})", url)
    if m: return m.group(1)
    if "v=" in url: return url.split("v=")[-1].split("&")[0]
    return url.replace("https://","").replace("http://","").replace("/","_")

def label_folder(base: str, split: str, label: int) -> str:
    d = f"{base}/{split}/{label}"
    Path(d).mkdir(parents=True, exist_ok=True)
    return d

def make_filename(entry: Dict) -> str:
    ytid = youtube_id_from_url(entry["url"])
    clean_text = str(entry.get("clean_text", "na")).replace(" ", "_")
    signer_id = entry.get("signer_id", "na")
    split = entry["split"]
    return f"{ytid}_{clean_text}_signer_id_{signer_id}_{split}.mp4"

def unique_id(entry: Dict) -> str:
    ytid  = youtube_id_from_url(entry["url"])
    start = entry.get("start_time", 0.0)
    end   = entry.get("end_time", 0.0)
    split = entry["split"]
    label = entry.get("label", -1)
    return f"{split}|{label}|{ytid}|{start}|{end}"

# =========================
# PROGRESS (mit Startwerten)
# =========================
class ProgressTracker:
    def __init__(self, progress_path: str, total: int, initial_done: int = 0, initial_failed: int = 0):
        self.path = progress_path
        self.total = total
        self.done = int(initial_done)
        self.failed = int(initial_failed)
        self.current_uid = None
        self.current_info = {}
        self.start_ts = time.time()
        self.window = deque(maxlen=50)
        self.window.append((self.start_ts, self.done + self.failed))
        self.hostname = socket.gethostname()
        self.lock = threading.Lock()

    def set_current(self, uid: str, info: Dict):
        with self.lock:
            self.current_uid = uid
            self.current_info = info
            self._write(status="running")

    def update_counts_and_write(self, success: bool):
        with self.lock:
            if success: self.done += 1
            else:       self.failed += 1
            now = time.time()
            processed = self.done + self.failed
            self.window.append((now, processed))
            self._write(status="running")

    def finish(self):
        with self.lock:
            self._write(status="finished")

    def _rate_eta(self):
        if len(self.window) < 2:
            return 0.0, None
        (t0, c0), (tn, cn) = self.window[0], self.window[-1]
        dt = max(1e-6, tn - t0)
        dc = max(0, cn - c0)
        rate_per_min = (dc / dt) * 60.0
        remaining = max(0, self.total - (self.done + self.failed))
        eta_min = (remaining / rate_per_min) if rate_per_min > 0 else None
        return rate_per_min, eta_min

    def _payload(self, status: str) -> Dict:
        now = time.time()
        rate, eta = self._rate_eta()
        return {
            "status": status,
            "last_update": utcnow_iso(),
            "start_time": datetime.fromtimestamp(self.start_ts, tz=timezone.utc).isoformat(),
            "uptime_sec": int(now - self.start_ts),
            "host": self.hostname,
            "total": self.total,
            "done": self.done,
            "failed": self.failed,
            "processed": self.done + self.failed,
            "remaining": max(0, self.total - (self.done + self.failed)),
            "rate_per_min": round(rate, 2),
            "eta_min": round(eta, 1) if eta is not None else None,
            "current": { "uid": self.current_uid, **self.current_info }
        }

    def _write(self, status: str):
        payload = self._payload(status)
        with state_lock:
            atomic_write_json(self.path, payload)

# =========================
# DOWNLOAD (voll lokal) + CUT (direkt Drive)
# =========================
def ytdlp_download_full_cli(url: str, tmp_path: str):
    Path(os.path.dirname(tmp_path)).mkdir(parents=True, exist_ok=True)
    cmd = [
        "yt-dlp",
        "-f", "bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4",
        "--retries", str(YTDLP_RETRIES),
        "--fragment-retries", str(FRAGMENT_RETRIES),
        "--concurrent-fragments", CONCURRENT_FRAGMENTS,
        "--http-chunk-size", HTTP_CHUNK_SIZE,
        "--merge-output-format", "mp4",
        "--no-warnings", "--no-call-home", "--restrict-filenames",
        "--geo-bypass",
        "-o", tmp_path,
        url
    ]
    if os.path.exists(COOKIES_PATH):
        cmd[1:1] = ["--cookies", COOKIES_PATH]
    if USE_PROXY and PROXY_URL:
        cmd[1:1] = ["--proxy", PROXY_URL]

    r = subprocess.run(cmd, capture_output=True, text=True)
    if r.returncode != 0:
        tail = "\n".join(((r.stderr or r.stdout or "").splitlines())[-20:])
        raise RuntimeError(f"yt-dlp failed (rc={r.returncode})\n{tail}")

def ffmpeg_cut(input_path: str, start: float, end: float, output_path: str) -> None:
    Path(os.path.dirname(output_path)).mkdir(parents=True, exist_ok=True)
    duration = None
    if start is not None and end is not None:
        duration = max(0.0, float(end) - float(start))
    if duration is not None and duration > 0:
        cmd = [
            "ffmpeg", "-hide_banner", "-loglevel", "error",
            "-ss", str(start), "-i", input_path, "-t", str(duration),
            "-c", "copy", "-movflags", "+faststart", "-y", output_path
        ]
        r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if r.returncode == 0 and os.path.exists(output_path) and os.path.getsize(output_path) > 0:
            return
        cmd = [
            "ffmpeg", "-hide_banner", "-loglevel", "error",
            "-ss", str(start), "-i", input_path, "-t", str(duration),
            "-c:v", "libx264", "-c:a", "aac", "-movflags", "+faststart",
            "-y", output_path
        ]
        r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if r.returncode != 0 or not os.path.exists(output_path):
            raise RuntimeError(r.stderr.decode(errors="ignore") or "ffmpeg failed")
    else:
        cmd = [
            "ffmpeg", "-hide_banner", "-loglevel", "error",
            "-i", input_path, "-c", "copy", "-movflags", "+faststart",
            "-y", output_path
        ]
        r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if r.returncode != 0 or not os.path.exists(output_path):
            raise RuntimeError(r.stderr.decode(errors="ignore") or "ffmpeg failed")

# =========================
# SCHÖNE AUSGABE
# =========================
def format_video_result(entry: Dict, success: bool) -> str:
    ytid = youtube_id_from_url(entry["url"])[:11]
    label_str = get_label_display(entry.get("label", -1))
    split = entry["split"].upper()
    clean_text = entry.get("clean_text", "Unknown")[:20]
    status = "✅" if success else "❌"
    return f"{status} {ytid} │ {split} │ Label {label_str} │ '{clean_text}' │ Gespeichert"

# =========================
# GROUP WORKER: eine YTID -> einmal laden, viele Schnitte
# =========================
def process_group(ytid: str, group_entries: List[Dict], processed: set, logger, tracker: ProgressTracker):
    # Nur Einträge schneiden, die wirklich fehlen
    to_cut = []
    for e in group_entries:
        uid = unique_id(e)
        final_dir = label_folder(DRIVE_ROOT, e["split"], int(e.get("label",-1)))
        final_out = f"{final_dir}/{make_filename(e)}"
        if os.path.exists(final_out):
            if uid not in processed:
                append_line(PROCESSED_FILE, uid)
                processed.add(uid)
                tracker.update_counts_and_write(True)
            print(f"⏩ bereits vorhanden: {os.path.basename(final_out)}")
            continue
        if uid in processed:
            print(f"⏩ bereits vorhanden: {os.path.basename(final_out)}")
            continue
        to_cut.append(e)

    if not to_cut:
        return 0, 0  # nichts zu tun

    # temp-Pfad pro YTID (keine Kollisionen mehr!)
    tmp_path = f"{TMP_YID_DIR}/{ytid}.mp4"
    # erste URL nehmen (alle gehören zur gleichen ID)
    url = _normalize_url(to_cut[0]["url"])

    # Volles Video ggf. laden (einmal pro Gruppe)
    if not os.path.exists(tmp_path) or os.path.getsize(tmp_path) == 0:
        try:
            safe_log(logger, f"⬇️  Lade voll: {ytid} → {tmp_path}")
            ytdlp_download_full_cli(url, tmp_path)
        except Exception as ex:
            # Wenn der Download schon hier scheitert, markiere alle Einträge als failed
            for e in to_cut:
                uid = unique_id(e)
                append_line(FAILED_FILE, uid)
                tracker.update_counts_and_write(False)
                safe_log(logger, f"❌ EXCEPTION {ytid}: {ex}", "error")
                print(f"❌ Fehler bei {uid}: {ex}")
            # aufräumen
            try:
                if os.path.exists(tmp_path):
                    os.remove(tmp_path)
            except:
                pass
            return 0, len(to_cut)

    ok_cnt, fail_cnt = 0, 0
    # Jetzt alle Clips der ID schneiden
    for e in to_cut:
        start_ts = float(e.get("start_time", 0.0))
        end_ts   = float(e.get("end_time", 0.0))
        if end_ts <= start_ts:
            end_ts = start_ts + 3.0
        if (end_ts - start_ts) > MAX_CLIP_LEN_S:
            end_ts = start_ts + MAX_CLIP_LEN_S

        final_dir = label_folder(DRIVE_ROOT, e["split"], int(e.get("label",-1)))
        final_out = f"{final_dir}/{make_filename(e)}"
        uid = unique_id(e)

        tracker.set_current(uid, {
            "ytid": ytid[:11], "split": e["split"], "label": int(e.get("label",-1)),
            "clean_text": str(e.get("clean_text", "Unknown"))[:50],
            "start": start_ts, "end": end_ts
        })

        print(f"▶ Verarbeite: {final_out}")
        safe_log(logger, f"▶ Verarbeite {uid} → {final_out}")

        try:
            ffmpeg_cut(tmp_path, start_ts, end_ts, final_out)
            append_line(PROCESSED_FILE, uid)
            tracker.update_counts_and_write(True)
            ok_cnt += 1
            msg = f"✅ erstellt: {os.path.basename(final_out)}"
            print(msg)
            safe_log(logger, msg)
        except Exception as ex:
            append_line(FAILED_FILE, uid)
            tracker.update_counts_and_write(False)
            fail_cnt += 1
            safe_log(logger, f"❌ EXCEPTION {ytid}: {str(ex)}", "error")
            print(f"❌ Fehler bei {uid}: {ex}")

    # tmp aufräumen (wir sind die einzigen, die diese YTID gerade bearbeiten)
    try:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
    except:
        pass

    return ok_cnt, fail_cnt

# =========================
# PIPELINE
# =========================
def process_all(logger: logging.Logger):
    ensure_dirs()
    load_label_names()

    entries   = read_selected50_entries(SELECTED_DIR)
    processed = read_lines(PROCESSED_FILE)
    failed    = read_lines(FAILED_FILE)

    # --- Backfill vor Worklist ---
    backfilled = 0
    for e in entries:
        uid = unique_id(e)
        final_path = f"{label_folder(DRIVE_ROOT, e['split'], e.get('label', -1))}/{make_filename(e)}"
        if os.path.exists(final_path) and uid not in processed:
            append_line(PROCESSED_FILE, uid)
            processed.add(uid)
            backfilled += 1
    safe_log(logger, f"🧾 Backfill processed_ids: +{backfilled} Einträge")

    total = len(entries)
    already_done = 0
    for e in entries:
        final_path = f"{label_folder(DRIVE_ROOT, e['split'], e.get('label', -1))}/{make_filename(e)}"
        if (unique_id(e) in processed) or os.path.exists(final_path):
            already_done += 1
    to_process = total - already_done

    # progress.json initial
    initial = {
        "status": "running" if to_process > 0 else "finished",
        "last_update": utcnow_iso(),
        "start_time": utcnow_iso(),
        "uptime_sec": 0,
        "host": socket.gethostname(),
        "total": total,
        "done": len(processed),
        "failed": len(failed),
        "processed": len(processed) + len(failed),
        "remaining": max(0, total - (len(processed)+len(failed))),
        "rate_per_min": 0.0,
        "eta_min": None,
        "current": {}
    }
    atomic_write_json(PROGRESS_JSON, initial)

    safe_log(logger, f"📊 Gefunden: {total} Videos total")
    safe_log(logger, f"📊 Bereits fertig: {already_done}")
    safe_log(logger, f"📊 Zu verarbeiten: {to_process}")
    safe_log(logger, "="*60)
    if to_process == 0:
        safe_log(logger, "🎉 Alle Videos bereits verarbeitet!")
        return

    tracker = ProgressTracker(
        PROGRESS_JSON,
        total=total,
        initial_done=len(processed),
        initial_failed=len(failed)
    )

    # --- Gruppieren nach YTID; nur fehlende in Gruppen packen ---
    groups = defaultdict(list)
    for e in entries:
        uid = unique_id(e)
        final_path = f"{label_folder(DRIVE_ROOT, e['split'], e.get('label', -1))}/{make_filename(e)}"
        if os.path.exists(final_path) or (uid in processed):
            continue
        ytid = youtube_id_from_url(e["url"])
        groups[ytid].append(e)

    # --- Parallel: pro YTID ein Job ---
    total_ok, total_fail = 0, 0
    if N_WORKERS <= 1:
        for ytid, group_entries in groups.items():
            ok, fail = process_group(ytid, group_entries, processed, logger, tracker)
            total_ok  += ok
            total_fail+= fail
    else:
        with ThreadPoolExecutor(max_workers=N_WORKERS) as ex:
            futs = [ex.submit(process_group, ytid, group_entries, processed, logger, tracker)
                    for ytid, group_entries in groups.items()]
            for fut in as_completed(futs):
                try:
                    ok, fail = fut.result()
                    total_ok  += ok
                    total_fail+= fail
                except Exception as e:
                    safe_log(logger, f"❌ FUTURE EXC (group): {e}", "error")

    safe_log(logger, "="*60)
    total_time = 1.0  # dummy für Rate; tracker hat die echte Rate
    safe_log(logger, f"🎯 FERTIG! (Gruppen abgeschlossen)")
    safe_log(logger, f"✅ Erfolgreich (neu erstellt): {total_ok}")
    safe_log(logger, f"❌ Fehlgeschlagen: {total_fail}")

    tracker.finish()

def retry_failed_videos(logger: logging.Logger):
    failed_ids = read_lines(FAILED_FILE)
    if not failed_ids:
        safe_log(logger, "🎉 Keine failed Videos zum Retry!")
        return
    safe_log(logger, f"🔄 Retry für {len(failed_ids)} gescheiterte Videos...")
    if os.path.exists(FAILED_FILE):
        os.remove(FAILED_FILE)
    process_all(logger)

# =========================
# RUN
# =========================
if __name__ == "__main__":
    logger = setup_logging()
    process_all(logger)


🚀 MSASL selected50 Downloader gestartet
INFO:msasl_downloader:🚀 MSASL selected50 Downloader gestartet
📁 Repo:      /content/sign-language-recognition
INFO:msasl_downloader:📁 Repo:      /content/sign-language-recognition
📂 Selected:  /content/sign-language-recognition/meta/selected50
INFO:msasl_downloader:📂 Selected:  /content/sign-language-recognition/meta/selected50
💾 Drive:     /content/drive/MyDrive/msasl_clips
INFO:msasl_downloader:💾 Drive:     /content/drive/MyDrive/msasl_clips
⚡ TempDir:   /content/msasl_tmp
INFO:msasl_downloader:⚡ TempDir:   /content/msasl_tmp
🍪 Cookies:   /content/sign-language-recognition/meta/selected50/cookies.txt (✅)
INFO:msasl_downloader:🍪 Cookies:   /content/sign-language-recognition/meta/selected50/cookies.txt (✅)
🧵 Workers:   2
INFO:msasl_downloader:🧵 Workers:   2
INFO:msasl_downloader:============================================================
🧾 Backfill processed_ids: +8 Einträge
INFO:msasl_downloader:🧾 Backfill processed_ids: +8 Einträge
📊 Gefunden:

▶ Verarbeite: /content/drive/MyDrive/msasl_clips/test/9/HBQaI1ZG-LU_deaf_signer_id_10_test.mp4
✅ erstellt: HBQaI1ZG-LU_deaf_signer_id_10_test.mp4
▶ Verarbeite: /content/drive/MyDrive/msasl_clips/test/9/HBQaI1ZG-LU_deaf_signer_id_10_test.mp4


✅ erstellt: HBQaI1ZG-LU_deaf_signer_id_10_test.mp4
INFO:msasl_downloader:✅ erstellt: HBQaI1ZG-LU_deaf_signer_id_10_test.mp4
⬇️  Lade voll: uE3IDWh0xnA → /content/msasl_tmp/_yid/uE3IDWh0xnA.mp4
INFO:msasl_downloader:⬇️  Lade voll: uE3IDWh0xnA → /content/msasl_tmp/_yid/uE3IDWh0xnA.mp4


✅ erstellt: HBQaI1ZG-LU_deaf_signer_id_10_test.mp4


▶ Verarbeite test|4|FBimL8-ND3E|61.795|63.263 → /content/drive/MyDrive/msasl_clips/test/4/FBimL8-ND3E_no_signer_id_101_test.mp4
INFO:msasl_downloader:▶ Verarbeite test|4|FBimL8-ND3E|61.795|63.263 → /content/drive/MyDrive/msasl_clips/test/4/FBimL8-ND3E_no_signer_id_101_test.mp4
✅ erstellt: FBimL8-ND3E_no_signer_id_101_test.mp4
INFO:msasl_downloader:✅ erstellt: FBimL8-ND3E_no_signer_id_101_test.mp4


...

✅ erstellt: 9bosgmeAAuo_again_signer_id_80_val.mp4
▶ Verarbeite: /content/drive/MyDrive/msasl_clips/val/26/9bosgmeAAuo_again_signer_id_80_val.mp4


✅ erstellt: 9bosgmeAAuo_again_signer_id_80_val.mp4
INFO:msasl_downloader:✅ erstellt: 9bosgmeAAuo_again_signer_id_80_val.mp4


✅ erstellt: 9bosgmeAAuo_again_signer_id_80_val.mp4


▶ Verarbeite val|4|URb48i4X52U|0.0|3.24 → /content/drive/MyDrive/msasl_clips/val/4/URb48i4X52U_no_signer_id_66_val.mp4
INFO:msasl_downloader:▶ Verarbeite val|4|URb48i4X52U|0.0|3.24 → /content/drive/MyDrive/msasl_clips/val/4/URb48i4X52U_no_signer_id_66_val.mp4
✅ erstellt: URb48i4X52U_no_signer_id_66_val.mp4
INFO:msasl_downloader:✅ erstellt: URb48i4X52U_no_signer_id_66_val.mp4
INFO:msasl_downloader:============================================================
🎯 FERTIG! (Gruppen abgeschlossen)
INFO:msasl_downloader:🎯 FERTIG! (Gruppen abgeschlossen)
✅ Erfolgreich (neu erstellt): 1823
INFO:msasl_downloader:✅ Erfolgreich (neu erstellt): 1823
❌ Fehlgeschlagen: 0
INFO:msasl_downloader:❌ Fehlgeschlagen: 0


▶ Verarbeite: /content/drive/MyDrive/msasl_clips/val/4/URb48i4X52U_no_signer_id_66_val.mp4
✅ erstellt: URb48i4X52U_no_signer_id_66_val.mp4
